In [1]:
import tkinter as tk
import PIL as PIL
from PIL import ImageGrab 
from tkinter import font


from PIL import Image, ImageFilter


def imageprepare(argv):
    """
    This function returns the pixel values.
    The imput is a png file location.
    """
    im = Image.open(argv).convert('L')
    width = float(im.size[0])
    height = float(im.size[1])
    
    
    newImage = Image.new('L', (28, 28), (0))  # creates white canvas of 28x28 pixels

    if width > height:  # check which dimension is bigger
        # Width is bigger. Width becomes 20 pixels.
        nheight = int(round((20.0 / width * height), 0))  # resize height according to ratio width
        if (nheight == 0):  # rare case but minimum is 1 pixel
            nheight = 1
            # resize and sharpen
        img = im.resize((20, nheight), Image.ANTIALIAS).filter(ImageFilter.SHARPEN)
       
        wtop = int(round(((28 - nheight) / 2), 0))  # calculate horizontal position
        newImage.paste(img, (4, wtop))  # paste resized image on white canvas
    else:
        # Height is bigger. Heigth becomes 20 pixels.
        nwidth = int(round((20.0 / height * width), 0))  # resize width according to ratio height
        if (nwidth == 0):  # rare case but minimum is 1 pixel
            nwidth = 1
            # resize and sharpen
        img = im.resize((nwidth, 20), Image.ANTIALIAS).filter(ImageFilter.SHARPEN)
        
        wleft = int(round(((28 - nwidth) / 2), 0))  # caculate vertical pozition
        newImage.paste(img, (wleft, 4))  # paste resized image on white canvas
    
    newImage.save("temp.png",dpi=(600,600))
    basewidth = 300
    img = Image.open("temp.png")
    wpercent = (basewidth / float(img.size[0]))
    hsize = int((float(img.size[1]) * float(wpercent)))
    img = img.resize((basewidth, hsize), PIL.Image.ANTIALIAS)
    img.save("temp.png",dpi=(600,600))
   


def predict_num(path):
 
    try:
        from tensorflow.keras.preprocessing import image
        import numpy as np
        from tensorflow.keras.models import model_from_json
        #load model
        json_file=open('model.json','r')
        loaded_model_json=json_file.read()
        json_file.close()
        loaded_model=model_from_json(loaded_model_json)
        loaded_model.load_weights("model.h5")
     

        loaded_model.compile(optimizer="adam",loss='sparse_categorical_crossentropy',metrics=['accuracy'])

        #score=loaded_model.evaluate(xtest,ytest)
        #print(score)
        test_image=image.load_img(path,target_size=(28,28),color_mode="grayscale")
        test_image.save("temp2.png")
        test_image=image.img_to_array(test_image)
        test_image=np.expand_dims(test_image,axis=0)
        test_image=test_image.astype('float32')
        test_image=test_image/255.0
        #print(test_image)
        #train_generator.class_indices

        result=loaded_model.predict(test_image)
      
        return result.argmax()

    except:
         print("invalid address")



def draw(event):
    x,y=event.x,event.y
    if canvas.old_coords:
        
        x1,y1=canvas.old_coords
        canvas.create_line(x,y,x1,y1,width=20,fill='white')
    canvas.old_coords=x,y
    
def rest_cords(event):
   
    canvas.old_coords=None    

def save_image():
    x=root.winfo_rootx()+canvas.winfo_x()
    y=root.winfo_rooty()+canvas.winfo_y()
    x1=x+250
    y1=y+250
    ImageGrab.grab().crop((x+5,y+5,x1,y1)).save("temp.png")
    x=tk.StringVar()
    imageprepare("temp.png")
    x=predict_num("temp.png")
  
    
     
    tk.Label(root,text=x,font=heading2,fg='#D6D6D6',bg='#222830').place(relx=0.7,rely=0.38) 
   
def clear_canvas():
    canvas.delete('all')

root=tk.Tk()
root.minsize(height=root.winfo_screenheight(),width=root.winfo_screenwidth())
root.configure(background='#222830')#2D142C 222830
root.title('predict handwritten number')
heading=font.Font(family='verdana',size=40,weight='bold')
heading2=font.Font(family='verdana',size=70,weight='bold')
content=font.Font(family='verdana',size=15,weight='bold')
tk.Label(root,text='NUMBER PREDICTOR',font=heading,fg='#D6D6D6',bg='#222830').place(relx=0.3,rely=0.1)
tk.Label(root,text="press and hold left mouse btn to draw",font=content,fg='red',bg='#222830').place(relx=0.0,rely=0.8)
tk.Label(root,text="RESULT:",font=heading,fg='#D6D6D6',bg='#222830').place(relx=0.5,rely=0.4) 

canvas=tk.Canvas(root,width=250,height=250,bg="black")
canvas.pack()
canvas.place(relx=0.1,rely=0.3)


btn=tk.Button(root,command=save_image,text="PREDICT",width="10",height="1",font=content,bg="#D6D6D6").place(rely=0.3,relx=0.3)
btn2=tk.Button(root,command=clear_canvas,text="CLEAR",width="10",height="1",font=content,bg="#D6D6D6").place(rely=0.58,relx=0.3)
canvas.old_coords=None
root.bind('<B1-Motion>',draw)
root.bind('<ButtonRelease-1>',rest_cords)

root.mainloop()